# <strong>SQL Injection</strong>

SQL (Structured Query Language) injection (abbreviated as SQLi) is a type of database attack by typing SQL code into an input field. SQL injection attacks can be very detrimental, as they can be used to leak information, change entries, wipe entire databases, and more. For this lab, you will be experimenting with a bank website that contains insecure code that does not sanitize user input, opening it up to security vulnerabilities.

<figure><center><img src="resources/sqli/sqli_example.jpg" style="width: 95%; height: 85%;"></img></center><em><figcaption>Credit: <a href="https://www.dnsstuff.com/sql-injection">DNSstuff</a></figcaption></em></figure>

The website where you will practice SQL injection is developed in PHP, which is hosted through Apache. PHP is a server-side language that's used for handling and processing data for websites. Since PHP is server-side, its code cannot be viewed by the user. However, this does not prevent attackers from attemping SQL injection, and you will learn in this lab how hackers can guess-and-check various SQL statements to crack a database.

<strong>This lab will contain four topics, and you will learn the following:</strong>

1. SQL Basics
2. PHP and SQL Practice
3. Prepared Statements
4. A Large-Scale Application of SQL Injection

In [31]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess
# For the stopexp command:
import re
# For later steps in Topic 4.
import shlex

# When true, it will not auto-save at each step.
runAllSteps = False

###### Used for saving notebooks. ######
import threading
# Threading required in case steps are progressed too quickly.
save_lock = threading.Lock()

# The save function itself.
def save_notebook():
    with save_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/save.py sqli"', shell=True, capture_output=True, text=True)

import threading, subprocess, shlex

def trigger_save(question, response, answer=""):
    save_thread = threading.Thread(target=save_notebook)
    save_thread.start()
    
    # Build a list of arguments for grader.py.
    cmd_args = ["/home/.education/grader.py", "S", question, response]
    if answer != "":
        cmd_args.append(answer)
    
    # Quote each argument separately (casting to string) and join them with a space.
    inner_cmd = " ".join(shlex.quote(str(arg)) for arg in cmd_args)
    
    # Now, pass this whole command to su using shlex.quote.
    full_cmd = f"su - USERNAME_GOES_HERE -c {shlex.quote(inner_cmd)}"
    result = subprocess.run(full_cmd, shell=True, capture_output=True, text=True)
    
###### Used for loading notebooks. ######
import queue

load_lock = threading.Lock()
result_queue = queue.Queue()

# This variable is used to warn students if they haven't loaded a save yet.
def warn_student():
    if (os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/.sqli_warning")):
        os.remove("/project/USERNAME_GOES_HERE/notebooks/saves/.sqli_warning")
        return True

def load_notebook():
    with load_lock:
        result = subprocess.run('su - USERNAME_GOES_HERE -c "/project/USERNAME_GOES_HERE/notebooks/resources/load.py sqli"', shell=True, capture_output=True, text=True)
        result_queue.put(result)  # Put the result in the queue.

# Creating a thread to load the notebook.
def trigger_load():
    load_thread = threading.Thread(target=load_notebook)
    load_thread.start()
    load_thread.join()  # Wait for the thread to complete before adding result to the queue.
    return result_queue.get()  # Get the result from the queue.

# This is used in Topic 2 after a successful payload:
def run_sql_query(payload):
    # Convert all double quotes to single quotes so that this will work. Cannot do \\' because SQL doesn't accept this.
    # Note: If this is Step 14, we cannot insert duplicate keys. So, we will just take the part of the payload that prints the table.
    payload = payload.replace('"', "'")
    payload = payload.split(';', 1)[0]

    # Send to the node.
    sql = f"SELECT student_id, student_name, student_grade FROM students WHERE student_id = {payload}"
    process = subprocess.run(
        f'ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "mysql -uroot -e \\"{sql}\\" practice"',
        shell=True, capture_output=True, text=True
    )

    # Return to the "create_markdown_table" function.
    return process.stdout

# This is used in Topic 2 after a successful payload:
def create_html_table(results):
    # Split the results into lines and process them.
    lines = results.strip().split("\n")
    
    # Extract the headers and rows.
    headers = lines[0].split("\t")
    rows = [line.split("\t") for line in lines[1:]]
    
    # Start HTML table
    html_table = "<table><thead><tr>"
    
    # Add header row
    for header in headers:
        html_table += f"<th>{header}</th>"
    html_table += "</tr></thead><tbody>"
    
    # Add data rows
    for row in rows:
        html_table += "<tr>"
        for cell in row:
            html_table += f"<td>{cell}</td>"
        html_table += "</tr>"
    
    # Close the table
    html_table += "</tbody></table>"
    
    return html_table

### Step 0: Starting the Lab

Click the button to begin creating the experiment.

<strong>Note:</strong> If your buttons are not displaying, click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to render all widgets.

In [2]:
# Click the button below to start the experiment.
import time

def check_autosave():
    # Check if the student has an autosave.
    if os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_sqli.tar.gz"):
        # Create a hidden file that will serve as a "boolean" for later.
        subprocess.run("touch /project/USERNAME_GOES_HERE/notebooks/saves/.sqli_warning", shell=True)

def startlab(button):
    # Defining the lab name.
    labname = "sqli"

    # Writing the information to an empty field below the button.
    with output0:
        # Fixes a strange error that happens only occasionally.
        os.chdir("/project/USERNAME_GOES_HERE/notebooks")
        output0.clear_output()

        # First, checking if the materialization exists. May have been stopped by a previous lab.
        materialPattern = "real." + labname + "jup.USERNAME_GOES_HERE"

        # Listing the materializations to find if there's an existing one for this lab.
        checkMaterial = os.popen('su - USERNAME_GOES_HERE -c "mrg list materializations"').read()
        regex = re.compile(materialPattern)
        # Getting the matches:
        match = regex.search(checkMaterial)

        if match:
            display(HTML("<span style='color: orange;'>An existing activation for this lab already exists. </span><span>You might have run another \
            lab without stopping this one. Attaching the existing activation...</span>"))

            # Detaching it and re-attaching.
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
            subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc.USERNAME_GOES_HERE real.' + labname + 'jup.USERNAME_GOES_HERE"', capture_output=True, text=True, shell=True, check=True)
            display(HTML("<span>Re-running the installation... </span><span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

            # Check if the student has an autosave.
            check_autosave()
            
            # Re-running the installation script.
            subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            output0.clear_output()
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been re-installed. </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook.</span>"))
        
        else:
            display(HTML("<span>No existing activations are found.</span>"))

            # Second, start the lab.
            display(HTML("<span>Starting the " + labname + " lab. This will take a few minutes to process. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            try:
                startexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /home/startexp ' + labname + 'jup"', capture_output=True, text=True, shell=True, check=True)
            except:
                output0.clear_output()
                display(HTML("<span style='color: red;'>There was an error starting your experiment. Make sure your password is written inside of ~/pass.txt, and try again.</span>"))
                return
            
            print(startexp.stdout, flush=True)
            output0.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(startexp.stdout, flush=True)

            # Another lab is already attached to the XDC.
            if ("XDC already attached" in startexp.stdout):
                existingLab = re.search(r"real.(.*).USERNAME_GOES_HERE", startexp.stdout).group(1)

                # Shouldn't happen.
                if labname == existingLab:
                    display(HTML("<span style='color: red;'>Your lab was already started. </span><span>Please continue to the next step.</span>"))

                # Detaching the existing lab, then attaching the current one.
                else:
                    display(HTML("<span style='color: orange;'>Warning: You did not stop your previous experiment. </span><span>Please stop your experiments \
                    before starting a new one. Detaching the " + existingLab + " experiment.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc detach xdc.USERNAME_GOES_HERE"', shell=True, check=True)
                    display(HTML("<span>Attaching the current lab.</span>"))
                    subprocess.run('su - USERNAME_GOES_HERE -c "mrg xdc attach xdc ' + materialPattern + '"', shell=True, check=True)
    
            # Third, get the lab materials onto the node.
            display(HTML("<span>Allocating lab resources onto the node. <u>Please wait a little longer...</u></span>"))

            # Gives the notebook a couple seconds so that it will recognize the node(s).
            time.sleep(2)

            # Move the resources over.
            runlab = subprocess.run(
                'su - USERNAME_GOES_HERE -c "bash /home/runlab ' + labname + 'jup"',
                capture_output=True, text=True, shell=True
            )

            # Check if the student has an autosave.
            check_autosave()

            # Complete. Inform the student.
            display(HTML("<newline><span style='color: green;'><strong>Setup complete. You may begin the lab! </strong></span>" \
                         "<span>When you're finished, close your lab at the bottom of the notebook. Your lab will be active for one week.</span>"))


# Creating the button.
startButton = widgets.Button(description="Start Lab")

# Creating an output area.
output0 = widgets.Output()

# Run the command on click.
startButton.on_click(startlab)

# Display the output.
display(startButton, output0)

Button(description='Start Lab', style=ButtonStyle())

Output()

<hr>

If you previously stopped your lab, you may restore your progress below by clicking "Load Lab". <u>You do not have to load your lab if you signed out, closed your notebook, or exited your node(s) or XDC by using ```exit```.</u>

In [3]:
# Click the button below to load your lab.
def loadlab(b):
    with output0_2:
        output0_2.clear_output()
        display(HTML("<span>Searching for an existing lab in your notebook...</span>"))

    if (os.path.exists("/project/USERNAME_GOES_HERE/notebooks/saves/USERNAME_GOES_HERE_sqli.tar.gz")):
        with output0_2:
            output0_2.clear_output()
            display(HTML("<span>Loading your lab...</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            result = trigger_load()
            if (result.returncode == 1):
                output0_2.clear_output()
                display(HTML("<span style='color: green;'>Your lab has been successfully loaded. Please click on the <img width='20px' height='20px' style='margin-left: 1px;' src='resources/fast_forward.png'> icon at the top of your notebook to reflect your changes.</span>"))
            elif (result.returncode == 2):
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>The sqli lab is inaccessible. Please start your lab. If you have already started it, wait a minute and try again.</span>"))
            else:
                output0_2.clear_output()
                display(HTML("<span style='color: red;'>An error occurred while loading your lab.</span>"))

# Creating the button.
loadButton = widgets.Button(description="Load Lab")

# Creating an output area.
output0_2 = widgets.Output()

# Run the command on click.
loadButton.on_click(loadlab)

# Display the output.
display(loadButton, output0_2)

Button(description='Load Lab', style=ButtonStyle())

Output()

## <strong>Topic 1: SQL Basics</strong>

For the first topic, you are going to learn the basics of SQL, such as creating a database, a table, and practicing queries like adding, updating, and deleting entries from the table.

Before practicing SQL, some important background and setup. SQL is a relational database management system (DBMS), meaning that data is stored as tables. Non-relational DBMS's exist, which is where data are stored as document files. An example of a non-relational DBMS is MongoDB. There are a few different variations of SQL which you may find online. A couple that you may find are MySQL, SQLite, PostgreSQL, SQL Server, and MariaDB. These all use SQL, but may have feature and performance differences. 

This lab is going to use MariaDB, which is backwards compatible with MySQL. This means that applications that use MySQL can easily switch to MariaDB without losing performance or features. MariaDB is designed to have more features, better performance, and fewer bugs than MySQL, and it's also more scalable and has faster query speeds. MariaDB has <a href="https://www.geeksforgeeks.org/difference-between-mysql-and-mariadb/">several extra features</a>, such as ```CREATE OR REPLACE TABLE``` and JSON query support. These features are advanced, and will not be covered in the lab.

You may view documentation called <a href="https://mariadb.com/kb/en/incompatibilities-and-feature-differences-between-mariadb-10-5-and-mysql-8-/">Incompatibilities and Feature Differences Between MariaDB 10.5 and MySQL 8.0</a> if you would like to learn more about MariaDB and how it's different from MySQL.

### Step 1: Create Your First Database

<strong>Sign into the ```sqli``` node.</strong>

MariaDB is already installed on your system, so you may access it by typing ```mariadb``` to open the interface. If you are more comfortable with MySQL, typing ```mysql``` will also bring up MariaDB since MariaDB is backwards-compatible, as mentioned before.

SQL statements are usually typed in UPPERCASE LETTERS, but is not required. However, to follow traditional syntax, queries will be written in UPPERCASE. <u>Additionally, SQL statements end with a semicolon.</u>

You can access MySQL by typing ```sudo mysql``` or ```mysql -uroot```. ```sudo mysql``` calls the "MySQL" statement as root. ```mysql -uroot``` calls the MySQL statement on your own account (USERNAME_GOES_HERE), but uses the username ```root```. There is no password for the ```root``` account, but if there would be a password, you would use ```mysql -uroot -ppassword```. There are no spaces between the ```-u``` (username) and ```-p``` (password) parameters.

First, list your databases by typing ```SHOW DATABASES;```. Currently, there are a couple databases, where one is called ```fccu```. <u>This is required for later in the lab, so at the moment, you will need to create another database.</u>

Use ```CREATE DATABASE practice;``` to create a database.

In [4]:
# Click the button below to check your work.
step1Complete = False

# Function to check the permissions.
def step_1():
    # Important variables that must be accessed outside of this function.
    global step1Complete, result

    with output1:
        output1.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/section_1.py 1 NA', shell=True)

    if (result.returncode == 0):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step1Complete = True
    
    elif (result.returncode == 1):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>No database named \"practice\" was found. Try again.</span>"))
            step1Complete = False
    
    elif (result.returncode == 2):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>An error occurred when checking your database. Please contact your professor or TA.</span>"))
            step1Complete = False

def check_step_1(b):
    if (warn_student()):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_1()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("1", result.returncode)

# Creating the button.
button = widgets.Button(description="Check For Database")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(button, output1)

Button(description='Check For Database', style=ButtonStyle())

Output()

### Step 2: Create Your First Table

You have created a database, and now you will need to "use" it. To select the database that you want to use, type ```USE practice;```. After using the ```USE``` command, you will need to create a table. Databases can have multiple tables to them, but you will only create one table for this lab.

We are going to create a table called ```students``` which will have three columns to them: ```student_id```, ```student_name```, and ```student_grade```. Each attribute has a type to them, similar to programming languages like C. Additionally, it's common for tables to have keys to them, which are unique identifiers for each entry. There are multiple types of keys, but we are going to use a ```PRIMARY KEY``` for the ```student_id``` attribute. A ```PRIMARY KEY``` is unique and cannot be a duplicate value. There are also candidate keys, super keys, alternate keys, foreign keys, and composite keys.

This is the layout of the table that you will create:
- ```student_id``` is of type ```SMALLINT UNSIGNED``` and a ```PRIMARY KEY```.
  - ```SMALLINT``` ranges between -32,768 to 32,767.
  - ```UNSIGNED``` takes the size of ```SMALLINT```, but only consists of positive values. Therefore, it will range from 0 to 65,535. We are using ```UNSIGNED``` so that there are no negative ID values.
  - <u>Note</u>: ```UNSIGNED``` is unsupported in PostgreSQL.
- ```student_name``` is of type ```VARCHAR(64)```
  - ```VARCHAR``` means "VARiable CHARacter", which dynamically allocates alphanumeric characters up to 64 characters.
  - This allows ```student_name``` to be 10 bytes in length if it's 10 characters long, up to 64 characters.
- ```student_grade``` is of type ```CHAR(1)```
  - ```CHAR``` is a fixed length of characters.
  - ```CHAR(15)``` will always take 15 bytes in memory, even if the string is not 15 characters long itself, like the word "database".
  - ```CHAR(1)``` is being used since students will be allocated a grade of A, B, C, D, or F.

The ```CREATE TABLE``` query can be lengthy, depending on the table. It consists of the following syntax: ```CREATE TABLE table_name (column1 datatype, column2 datatype, ...);```

Given the following columns and datatypes, create a table called ```students``` that has the following:
- ```student_id SMALLINT UNSIGNED PRIMARY KEY```
- ```student_name VARCHAR(64)```
- ```student_grade CHAR(1)```

<span style="color: orange;"><strong><img src="resources/alert.png" style="width: 12px"> Notice:</strong></span> All SQL commands end with semicolons. If you press "Enter" without finishing your statement with ```;```, it will continue to make a new line until you terminate it with a ```;```.

In [5]:
# Click the button below to check your work.
step2Complete = False

# Function to check the permissions.
def step_2():
    # Important variables that must be accessed outside of this function.
    global step2Complete, result

    with output2:
        output2.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/section_1.py 2 NA', shell=True)

    if (result.returncode == 0):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True

    elif (result.returncode == 2):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
            step2Complete = False

    elif (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>The \"students\" table exists, but the datatypes are incorrect. Make sure you specifically create the columns from above.</span>"))
            step2Complete = False

    elif (result.returncode == 3):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>There was no table found named \"students\" inside of the \"practice\" database.</span>"))
            step2Complete = False

def check_step_2(b):
    if (warn_student()):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_2()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("2", result.returncode)

# Creating the button.
button = widgets.Button(description="Check For Table")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check For Table', style=ButtonStyle())

Output()

### Step 3: Inserting SQL Entries

To add SQL entries into the table, use the ```INSERT INTO``` command. The syntax is: ```INSERT INTO table (column1, column2, ...) VALUES (A, B, ...), (C, D, ...), ...;```

These are the following entries that you will need to add into the table:
- ID: 100, Name: Aaron, Grade: A
- ID: 101, Name: Danny, Grade: B
- ID: 102, Name: Hannah, Grade: D
- ID: 103, Name: Abby, Grade: ```NULL```

When using ```INSERT```, the ```INT``` datatype does not require quotes, but ```CHAR``` or ```VARCHAR``` require quotes around them (similar to strings in programming languages).

One of the entries contains a ```NULL``` value. SQL allows ```NULL``` values, but these are not equal to ```0``` or ```"NULL"```. <strong>```NULL``` is not a string</strong>. ```NULL``` is a variable itself to describe "no data". When you assign something to be ```NULL``` in SQL, it's done by writing ```INSERT INTO (...) VALUES (..., NULL)```.

Add the four entries to the ```students``` table.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>

- Order does not matter when using ```INSERT INTO```. If you created your table as ```student_id```, ```student_grade```, then ```student_name```, you may use ```INSERT INTO students (student_id, student_name, student_grade), VALUES (...)```, where ```student_name``` and ```student_grade``` are in opposite order.
- Not all DBMS's support double quotes. Some SQL modes only support single quotes. Fortunately, MariaDB supports both single quotes and double quotes. Backticks (``` ` ```) can be used to identify columns, though this is optional. However, it may be required if you make a (poor) decision naming an attribute after a reserved keyword, like ``` `column` ```.
- As emphasized before, ```NULL``` is not a string. Consider this when adding your fourth entry. Integers cannot have quotes around them, and strings must have quotes around them.
- <strong>If you make a mistake and want to delete a row</strong>, you may look ahead at Step 8, which describes the ```DELETE``` keyword.

In [6]:
# Click the button below to check your work.
step3Complete = False

# Function to check the permissions.
def step_3():
    # Important variables that must be accessed outside of this function.
    global step3Complete, step12Complete, result

    with output3:
        output3.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/section_1.py 3 NA', shell=True)

    # Checking to see if step 12 was completed, because if it is, then this step was previously passed.
    if (result.returncode == 0 or step12Complete):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

    elif (result.returncode == 1):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>The four rows that were provided either cannot be found or have an incorrect value. Ensure that the only rows in your database are the ones listed above.</span>"))
            step3Complete = False
    
    elif (result.returncode == 2):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
            step3Complete = False

def check_step_3(b):
    if (warn_student()):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_3()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("3", result.returncode)

# Creating the button.
button = widgets.Button(description="Check For Table")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check For Table', style=ButtonStyle())

Output()

### Step 4: Practicing SQL Queries (Part 1)

You now have a table that's populated with entries. Next, you are going to perform some SQL statements that will return rows from your table.

This is an example of a SQL query: ```SELECT * FROM table WHERE condition```

- ```SELECT *``` returns all columns. You are able to select certain columns of a table, like ```SELECT student_id```. 
- The ```FROM``` clause selects the table that you wish to grab entries from.
- The ```WHERE``` clause takes a condition that returns entries that match it.

Using the text prompt below, type a SQL statement that will print everything in the table. You do not need to use a ```WHERE``` clause for your query.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span> 
- You are encouraged to try running this in MariaDB on your ```sqli``` node first. When your answer is ready, you may copy your SQL query into the entry box.
- <u>It's important to know the order in which you type SQL clauses.</u> You are not allowed to interchange the ```SELECT```, ```FROM```, and ```WHERE``` clauses. They must be in this order.
- Remember that all SQL statements end with a semicolon.

<strong>The remaining steps throughout this section will contain a "Reset" button.</strong> If you made an error to your database, and wish to revert your database to the previous step, click on the "Reset" button.

In [7]:
# Click the button below to check your work.
step4Complete = False

# Function to check if the student's answer was correct.
def step_4():
    # Important variables that must be accessed outside of this function.
    global step4Complete, result

    # Loading, in case the check is slow.
    with output4:
        output4.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput4.value == ""):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step4Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput4.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_1.py 4 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_4_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step4Complete = True
        
        elif (result.returncode == 1):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step4Complete = False
        
        elif (result.returncode == 2):
            output4.clear_output()
            with output4:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step4Complete = False

def check_step_4(b):
    if (warn_student()):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_4()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("4", result.returncode, userInput4.value)

def reset_step_4(b):
    if (not step3Complete or step4Complete):
        output4.clear_output()
        with output4:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 4', shell=True, stdout=subprocess.DEVNULL)
    
        output4.clear_output()
        with output4:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))

    
# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading4 = widgets.Output()
display(loading4)
with loading4:
    loading4.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput4 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_4_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput4.value = result.stdout

# After the student's response was loaded, clear the output.
loading4.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_4 = widgets.Button(description="Reset Database")

# Creating an output area.
output4 = widgets.Output()

# Run the command on click.
button.on_click(check_step_4)
button_step_4.on_click(reset_step_4)

# Display the output.
display(userInput4, button, button_step_4, output4)

Output()

Text(value='', description='Query:', layout=Layout(width='90%'), placeholder='Type your SQL query here')

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 5: Practicing SQL Queries (Part 2)

The ```WHERE``` clause takes different types of conditions. Here are some examples:
- ```WHERE id < 200```
- ```WHERE name != "Adam"```
  - ```<>``` is another operator for "not equal" in MariaDB.
- ```WHERE city LIKE "San %"```
  - The ```%``` is a wildcard, where any length of characters are accepted.
  - The ```_``` is a wildcard for a single character. Using two ```_```'s will match two random characters.

Using the ```WHERE``` clause, type a SQL statement that will return all students that have an "A" for a grade.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> Your ```WHERE``` statement must consist of matching one of your columns with a ```VARCHAR``` (a string). Recall that comparing strings in SQL need "quotes" around them. You may use either single quotes or double quotes.

In [8]:
# Click the button below to check your work.
step5Complete = False

# Function to check if the student's answer was correct.
def step_5():
    # Important variables that must be accessed outside of this function.
    global step5Complete, result

    # Loading, in case the check is slow.
    with output5:
        output5.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput5.value == ""):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step5Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput5.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_1.py 5 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_5_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step5Complete = True
        
        elif (result.returncode == 1):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step5Complete = False
        
        elif (result.returncode == 2):
            output5.clear_output()
            with output5:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step5Complete = False

def check_step_5(b):
    if (warn_student()):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_5()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("5", result.returncode, userInput5.value)

def reset_step_5(b):
    global step4Complete, step5Complete
    if (not step4Complete or step5Complete):
        output5.clear_output()
        with output5:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 5', shell=True, stdout=subprocess.DEVNULL)
    
        output5.clear_output()
        with output5:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading5 = widgets.Output()
display(loading5)
with loading5:
    loading5.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput5 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_5_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput5.value = result.stdout

# After the student's response was loaded, clear the output.
loading5.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_5 = widgets.Button(description="Reset Database")

# Creating an output area.
output5 = widgets.Output()

# Run the command on click.
button.on_click(check_step_5)
button_step_5.on_click(reset_step_5)

# Display the output.
display(userInput5, button, button_step_5, output5)

Output()

Text(value='', description='Query:', layout=Layout(width='90%'), placeholder='Type your SQL query here')

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 6: Practicing SQL Queries (Part 3)

Type a SQL statement that returns all students who do not have a ```NULL``` grade.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> Recall that ```NULL``` is not a string, therefore it cannot be tested like the previous step. Instead, use ```IS NOT NULL``` in your ```WHERE``` statement.

In [9]:
# Click the button below to check your work.
step6Complete = False

# Function to check if the student's answer was correct.
def step_6():
    # Important variables that must be accessed outside of this function.
    global step6Complete, result

    # Loading, in case the check is slow.
    with output6:
        output6.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput6.value == ""):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step6Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput6.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_1.py 6 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_6_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step6Complete = True
        
        elif (result.returncode == 1):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step6Complete = False
        
        elif (result.returncode == 2):
            output6.clear_output()
            with output6:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step6Complete = False

def check_step_6(b):
    if (warn_student()):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_6()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("6", result.returncode, userInput6.value)

def reset_step_6(b):
    if (not step5Complete):
        output6.clear_output()
        with output6:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 6', shell=True, stdout=subprocess.DEVNULL)
    
        output6.clear_output()
        with output6:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading6 = widgets.Output()
display(loading6)
with loading6:
    loading6.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput6 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_6_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput6.value = result.stdout

# After the student's response was loaded, clear the output.
loading6.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_6 = widgets.Button(description="Reset Database")

# Creating an output area.
output6 = widgets.Output()

# Run the command on click.
button.on_click(check_step_6)
button_step_6.on_click(reset_step_6)

# Display the output.
display(userInput6, button, button_step_6, output6)

Output()

Text(value='', description='Query:', layout=Layout(width='90%'), placeholder='Type your SQL query here')

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 7: Practicing SQL Queries (Part 4)

SQL rows can be updated by using the ```UPDATE``` command.

The syntax works as follows: ```UPDATE table SET column1 = value1, column2 = value2, ... WHERE condition;```

The ```WHERE``` clause is optional within the ```UPDATE``` command. However, when not matching a condition, it will change all values in a given column to a certain amount. Therefore, it's practical to use ```WHERE``` when working with the ```UPDATE``` command.

For example, if you didn't use ```WHERE```, the command ```UPDATE students SET student_name = 'Harry';``` will change everyone's name to Harry.

Type a SQL statement that changes Aaron's grade to a B.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> If you need to revert your database back to what it originally was, you may click on the "Reset" button. This will add all entries back to your database again.

In [10]:
# Click the button below to check your work.
step7Complete = False

# Function to check if the student's answer was correct.
def step_7():
    # Important variables that must be accessed outside of this function.
    global step7Complete, result

    # Loading, in case the check is slow.
    with output7:
        output7.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput7.value == ""):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step7Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput7.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_1.py 7 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_7_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step7Complete = True
        
        elif (result.returncode == 1):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step7Complete = False
        
        elif (result.returncode == 2):
            output7.clear_output()
            with output7:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step7Complete = False

def check_step_7(b):
    if (warn_student()):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_7()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("7", result.returncode, userInput7.value)

def reset_step_7(b):
    if (not step6Complete):
        output7.clear_output()
        with output7:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 7', shell=True, stdout=subprocess.DEVNULL)
    
        output7.clear_output()
        with output7:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading7 = widgets.Output()
display(loading7)
with loading7:
    loading7.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput7 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_7_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput7.value = result.stdout

# After the student's response was loaded, clear the output.
loading7.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_7 = widgets.Button(description="Reset Database")

# Creating an output area.
output7 = widgets.Output()

# Run the command on click.
button.on_click(check_step_7)
button_step_7.on_click(reset_step_7)

# Display the output.
display(userInput7, button, button_step_7, output7)

Output()

Text(value='', description='Query:', layout=Layout(width='90%'), placeholder='Type your SQL query here')

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 8: Practicing SQL Queries (Part 5)

SQL rows can also be removed by using the ```DELETE``` command.

The syntax for ```DELETE``` is: ```DELETE FROM table WHERE condition;```

Type a SQL statement that removes everyone whose grade is NULL.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> If you need to revert your database, you may click on the "Reset" button. Since you have completed Step 8, it will not reverse your progress from the previous step.

In [11]:
# Click the button below to check your work.
step8Complete = False

# Function to check if the student's answer was correct.
def step_8():
    # Important variables that must be accessed outside of this function.
    global step8Complete, result

    # Loading, in case the check is slow.
    with output8:
        output8.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput8.value == ""):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step8Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput8.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_1.py 8 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_8_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step8Complete = True
        
        elif (result.returncode == 1):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step8Complete = False
        
        elif (result.returncode == 2):
            output8.clear_output()
            with output8:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step8Complete = False

def check_step_8(b):
    if (warn_student()):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_8()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("8", result.returncode, userInput8.value)

def reset_step_8(b):
    if (not step7Complete):
        output8.clear_output()
        with output8:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 8', shell=True, stdout=subprocess.DEVNULL)
    
        output8.clear_output()
        with output8:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading8 = widgets.Output()
display(loading8)
with loading8:
    loading8.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput8 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_8_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput8.value = result.stdout

# After the student's response was loaded, clear the output.
loading8.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_8 = widgets.Button(description="Reset Database")

# Creating an output area.
output8 = widgets.Output()

# Run the command on click.
button.on_click(check_step_8)
button_step_8.on_click(reset_step_8)

# Display the output.
display(userInput8, button, button_step_8, output8)

Output()

Text(value='', description='Query:', layout=Layout(width='90%'), placeholder='Type your SQL query here')

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 9: Intermediate SQL Queries - (OPTIONAL)

Additional clauses may be used in SQL queries for more advanced searches. A couple of useful clauses are: ```LIMIT```, ```OFFSET```, ```ORDER BY```, and ```GROUP BY```. There are more clauses than this, but a couple of these clauses might be helpful when doing Topic 4 in this lab.

- ```LIMIT``` takes the entries that your query returns, and only returns the top X rows, where X is the integer value from ```LIMIT X```.
- ```OFFSET``` is similar to ```LIMIT```, but skips the first X rows where X is the integer value from ```OFFSET X```.
  - ```OFFSET``` is useful when implementing previous page and next page functionality for websites.
  - ```OFFSET``` cannot be used without the ```LIMIT``` clause.
  - ```OFFSET``` cannot be called before ```LIMIT```.
- ```ORDER BY``` takes a column from your result query and sorts it numerically/alphabetically.
  - ```ASC``` and ```DESC``` are optional after the ```ORDER BY``` statement, which stand for "ascending" and "descending".
- ```GROUP BY``` is similar to ```ORDER BY```, except it is mainly used for aggregation functions, like ```COUNT()```, ```MAX()```, ```MIN()```, ```SUM()```, and ```AVG()```. These aggregation functions are placed in the ```SELECT``` statement of SQL queries, and return numerical values.
  - An example of ```GROUP BY``` is: ```SELECT COUNT(student_grade) FROM students WHERE student_grade == 'A' GROUP BY student_grade;```, which returns the number of students that have an A for a grade.
  - You cannot call these aggregation functions without ```GROUP BY```.
 
For your task, create a SQL statement that returns all entries in the table, but prints only the first row and offset your result by one.

In [12]:
# Click the button below to check your work.
step9Complete = False

# Function to check if the student's answer was correct.
def step_9():
    # Important variables that must be accessed outside of this function.
    global step9Complete, result

    # Loading, in case the check is slow.
    with output9:
        output9.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput9.value == ""):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step9Complete = False

    else:
        # Escape single quotes by replacing them with `'\''`.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput9.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_1.py 9 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_9_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: green;'>Your SQL query is correct!</span>"))
                step9Complete = True
        
        elif (result.returncode == 1):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: red;'>Your SQL query is invalid or incorrect. Try again.</span>"))
                step9Complete = False
        
        elif (result.returncode == 2):
            output9.clear_output()
            with output9:
                display(HTML("<span style='color: red;'>An error occurred when checking your database. Make sure you have completed the previous step(s). Otherwise, please contact your professor or TA.</span>"))
                step9Complete = False

def check_step_9(b):
    if (warn_student()):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_9()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("9", result.returncode, userInput9.value)

def reset_step_9(b):
    if (not step8Complete):
        output9.clear_output()
        with output9:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 9', shell=True, stdout=subprocess.DEVNULL)
    
        output9.clear_output()
        with output9:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading9 = widgets.Output()
display(loading9)
with loading9:
    loading9.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput9 = widgets.Text(
    placeholder='Type your SQL query here',
    description='Query:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_9_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput9.value = result.stdout

# After the student's response was loaded, clear the output.
loading9.clear_output()

# Creating the button.
button = widgets.Button(description="Run SQL Query")

# Create an extra button for resetting the database.
button_step_9 = widgets.Button(description="Reset Database")

# Creating an output area.
output9 = widgets.Output()

# Run the command on click.
button.on_click(check_step_9)
button_step_9.on_click(reset_step_9)

# Display the output.
display(userInput9, button, button_step_9, output9)

Output()

Text(value='', description='Query:', layout=Layout(width='90%'), placeholder='Type your SQL query here')

Button(description='Run SQL Query', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

## <strong>Topic 2: PHP and SQL Practice</strong>

PHP, as mentioned before, is a programming language to handle server-side processing. It's used in back-end website development, which is everything that the website users cannot see. Front-end web development is everything that the users of a website can see, like HTML, CSS, and JavaScript.

Since PHP is back-end, it will hide SQL statements from the user, which provides some security for websites. However, when focusing on computer security, you cannot assume that the hacker doesn't know the system. For this topic, you will learn how SQL statements are called from PHP, which take user input from HTML forms. You are going to view a sample of unsafe PHP code that can put your database at risk. Additionally, you will learn how to break an insecure system.

<strong>This portion of the lab is going to require port forwarding, which requires setup on your own, local machine. If you have not already done port forwarding, you will need to follow the steps from this notebook.</strong>

### Step 10: Accessing ```php_practice.php```

After you have port forwarded to your ```sqli``` node, access ```localhost:port/php_practice.php```, where ```port``` is the number that you used in your SSH statement.

<strong>For example</strong>, if you have already set up port forwarding, type the following command into your terminal: ```ssh -L 8080:sqli:80 USERNAME_GOES_HERE-xdc-USERNAME_GOES_HERE```, then access the practice page by typing ```http://localhost:8080/php_practice.php``` into your web browser.

The website's source code is located at ```/var/www/html/php_practice.php```.

This website contains a simple form which takes a student's ID. When you type in a student's ID into the field, it takes your input, puts it into a SQL statement, sends the SQL statement to the database, retrieve the results, then it displays it on the website. <u>You may test this by typing 100 into the Student ID field.</u>

For your convenience and to help demonstrate the lab, the SQL statement that's being sent to the database will display on the page for you, highlighting the statement that you typed in. Feel free to use this to assist your understanding of SQL injection, as it will be unavailable for the large scale portion of the lab.

### Step 11: Observe the PHP File

Now, open ```/var/www/html/php_practice.php``` in your ```sqli``` node, and observe the file contents. Look for the SQL query, and observe how the user input is used in the SQL statement. Additionally, notice that ```mysqli_multi_query()``` is being used to run the query from the input field.

In the text field below, explain the possible security vulnerability with this approach.

In [13]:
# Click the button below to check your work.
step11Complete = False

# Function to save the short answer.
def step_11():
    # Important variables that must be accessed outside of this function.
    global step11Complete, result

    # Loading, in case saving is slow.
    with output11:
        output11.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    if (userInput11.value == ""):
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'>You did not type a response.</span>"))
            step11Complete = False

    else:
        # Replace backticks with single quotes in the input.
        safe_value = userInput11.value.replace("`", "'")
        
        # Build the SSH command to write to the file using 'cat' reading from stdin.
        ssh_command = [
            "ssh",
            "-i", "/home/USERNAME_GOES_HERE/.ssh/merge_key",
            "USERNAME_GOES_HERE@sqli",
            "cat > /home/.checker/responses/step_11_answer.txt"
        ]
        
        # Pass safe_value as input so that no shell quoting is needed.
        result = subprocess.run(ssh_command, input=safe_value, text=True)
        
        if result.returncode == 0:
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: red;'>There was an error saving your response.</span>"))
                step11Complete = False
                
        elif result.returncode == 1:
            output11.clear_output()
            with output11:
                display(HTML("<span style='color: green;'>Your response was saved.</span>"))
                step11Complete = True
                

def check_step_11(b):
    if warn_student():
        output11.clear_output()
        with output11:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_11()
        if not runAllSteps:
            safe_value = userInput11.value.replace("`", "'")
            user_input_quoted = shlex.quote(safe_value)
            trigger_save("11", 1 if result.returncode == 1 else 0, user_input_quoted)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading11 = widgets.Output()
display(loading11)
with loading11:
    loading11.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area. We will need to assign output of process to the value of this input.
userInput11 = widgets.Textarea(
    placeholder='Type your response here',
    description='Response:',
    layout=widgets.Layout(width='75%', height='150px', margin='10px')
)

def on_input_change(change):
    # Replace backticks with single quotes in the new value.
    new_val = change['new'].replace("`", "'")
    # Only update if there's a change (to avoid unnecessary recursion).
    if new_val != change['new']:
        userInput11.value = new_val

userInput11.observe(on_input_change, names='value')

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_11_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
# Output creates a newline. Remove it.
userInput11.value = result.stdout[:-1]

# After the student's response was loaded, clear the output.
loading11.clear_output()

# Creating the feedback output area.
output11 = widgets.Output()

# Creating the button.
button = widgets.Button(description="Save Response")

# Run the command on click.
button.on_click(check_step_11)

# Display the output.
display(userInput11, button, output11)

Output()

Textarea(value='', description='Response:', layout=Layout(height='150px', margin='10px', width='75%'), placeho…

Button(description='Save Response', style=ButtonStyle())

Output()

### Step 12: Practicing SQL Injection (Part 1)

In the previous step, you should've observed the security vulnerability with this PHP code. From Step 10, you observed that typing a valid student ID works, but there is are some security issues with this setup.

<strong>Your task:</strong> Using this to your advantage, type SQL code into the "Student ID" field on the website to change Aaron's name to Taylor. 

After testing your statement, you will need to copy/paste your input into the field below to check your work. Your database will be reset and your query will be tested to determine if your SQL statement is correct. If you would like to reset your database, you may click on the "Reset" button, similar to Section 1.

<u>If you're lost with where to begin</u>, look at the ```$sql``` variable in the source code: ```SELECT student_id, student_name, student_grade FROM students WHERE student_id = $student_id```. Understand that ```$student_id``` is the value from the input field. Consider this: ```student_id``` needs to be equal to something, but if you use ```OR``` in your statement, then typing another comparison that's <u>always</u> true will create no filtering. Thus, printing everything from ```students```.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> If you haven't already, try a very basic SQL injection that prints all database entries in the table. If you know how to do this basic SQL injection attack, you can end it with a ```;```, then type in the payload that will change Aaron to Taylor. This works because the PHP file is using ```mysqli_multi_query```, which allows you to process multiple SQL queries in one statement. The PHP file is setup like this so that you can run an ```UPDATE``` command within your payload. If ```mysqli_query``` were to be used instead, your payload would not work. It would only accept one SQL query.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> When you perform your SQL injection attack, your ```UPDATE``` statement comes after a ```SELECT``` statement, which prints your table. This means that your changes <u>will not appear reflected until you run another ```SELECT``` statement.</u> If you think your payload worked, try running your payload again to see if your database updated. Or, check your database within MySQL on ```sqli``` to see if the database updated.

In [14]:
# Click the button below to check your work.
step12Complete = False

# Function to check if the student's answer was correct.
def step_12():
    # Important variables that must be accessed outside of this function.
    global step12Complete, result

    # Loading, in case the check is slow.
    with output12:
        output12.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput12.value == ""):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step12Complete = False

    else:
        # Escape single quotes by replacing them with '\''.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput12.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_2.py 12 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_12_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: green;'>Your payload is correct! After sending your payload to the website, it produced the table below in your SQL database.</span>"))
                step12Complete = True
        
        elif (result.returncode == 1):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: red;'>Your payload didn't work. Try again.</span>"))
                step12Complete = False
        
        elif (result.returncode == 2):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: red;'>An error occurred while testing your payload. Please try again or contact your professor or TA.</span>"))
                step12Complete = False

        elif (result.returncode == 3):
            output12.clear_output()
            with output12:
                display(HTML("<span style='color: green;'>You have completed this step previously, and your file is patched.</span>"))
                step12Complete = True

        # Now, if the return code was 0 or 1, it returned a response. Print the HTML of the response:
        if (result.returncode == 0 or result.returncode == 1):
            result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_12_response.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
            with output12:
                if ("No results found. Please enter a valid ID." in result.stdout):
                    display(HTML("<br><p style='color: blue;'><em>Your php_practice.php file has its SQL injection vulnerability fixed.</em></p>"))
                
                else:
                    display(HTML("<br><p style='color: blue;'><em>Here is your SQL database after sending your payload:</em></p>"))
                    results = run_sql_query(userInput12.value)

                    if results:
                        markdown = create_html_table(results)
                        display(HTML(markdown))

                    else:
                        display(HTML("<p>The website did not produce any output.</p>"))

def check_step_12(b):
    if (warn_student()):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_12()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("12", result.returncode, userInput12.value)

def reset_step_12(b):
    if (not step11Complete):
        output12.clear_output()
        with output12:
            display(HTML("<span style='color: red;'>You must complete Step 11 before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 12', shell=True, stdout=subprocess.DEVNULL)
    
        output12.clear_output()
        with output12:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))


# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading12 = widgets.Output()
display(loading12)
with loading12:
    loading12.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput12 = widgets.Text(
    placeholder='Type your payload here',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_12_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput12.value = result.stdout

# After the student's response was loaded, clear the output.
loading12.clear_output()

# Creating the button.
button = widgets.Button(description="Test Payload")

# Create an extra button for resetting the database.
button_step_12 = widgets.Button(description="Reset Database")

# Creating an output area.
output12 = widgets.Output()

# Run the command on click.
button.on_click(check_step_12)
button_step_12.on_click(reset_step_12)

# Display the output.
display(userInput12, button, button_step_12, output12)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type your payload here')

Button(description='Test Payload', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 13: Practicing SQL Injection (Part 2)

Now, using the input field on the website, change Hannah's grade to a B.

Once you have a working command, type the command below and check your work. Your database will be reverted back to completing Step 12, and it will be tested. Again, a "Reset" button is available for you, and it will not reset your progress from the previous step.

In [15]:
# Click the button below to check your work.
step13Complete = False

# Function to check if the student's answer was correct.
def step_13():
    # Important variables that must be accessed outside of this function.
    global step13Complete, result

    # Loading, in case the check is slow.
    with output13:
        output13.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput13.value == ""):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step13Complete = False

    else:
        # Escape single quotes by replacing them with '\''.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput13.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_2.py 13 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_13_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output13.clear_output()
            with output13:
                display(HTML("<span style='color: green;'>Your payload is correct! After sending your payload to the website, it produced the table below in your SQL database.</span>"))
                step13Complete = True
        
        elif (result.returncode == 1):
            output13.clear_output()
            with output13:
                display(HTML("<span style='color: red;'>Your payload didn't work. Try again.</span>"))
                step13Complete = False
        
        elif (result.returncode == 2):
            output13.clear_output()
            with output13:
                display(HTML("<span style='color: red;'>An error occurred while testing your payload. Please try again or contact your professor or TA.</span>"))
                step13Complete = False

        elif (result.returncode == 3):
            output13.clear_output()
            with output13:
                display(HTML("<span style='color: green;'>You have completed this step previously, and your file is patched.</span>"))
                step13Complete = True

        # Now, if the return code was 0 or 1, it returned a response. Print the HTML of the response:
        if (result.returncode == 0 or result.returncode == 1):
            result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_13_response.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
            with output13:
                if ("No results found. Please enter a valid ID." in result.stdout):
                    display(HTML("<br><p style='color: blue;'><em>Your php_practice.php file has its SQL injection vulnerability fixed.</em></p>"))
                
                else:
                    display(HTML("<br><p style='color: blue;'><em>Here is your SQL database after sending your payload:</em></p>"))
                    results = run_sql_query(userInput13.value)

                    if results:
                        markdown = create_html_table(results)
                        display(HTML(markdown))

def check_step_13(b):
    if (warn_student()):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_13()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("13", result.returncode, userInput13.value)

def reset_step_13(b):
    if (not step11Complete):
        output13.clear_output()
        with output13:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 13', shell=True, stdout=subprocess.DEVNULL)
    
        output13.clear_output()
        with output13:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))


# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading13 = widgets.Output()
display(loading13)
with loading13:
    loading13.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput13 = widgets.Text(
    placeholder='Type your payload here',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_13_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput13.value = result.stdout

# After the student's response was loaded, clear the output.
loading13.clear_output()

# Creating the button.
button = widgets.Button(description="Test Payload")

# Create an extra button for resetting the database.
button_step_13 = widgets.Button(description="Reset Database")

# Creating an output area.
output13 = widgets.Output()

# Run the command on click.
button.on_click(check_step_13)
button_step_13.on_click(reset_step_13)

# Display the output.
display(userInput13, button, button_step_13, output13)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type your payload here')

Button(description='Test Payload', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

### Step 14: Practicing SQL Injection (Part 3)

Finally, using the input field, create a student named Jason whose ID is 200 and has an F for a grade.

A "Reset" button is available to revert to the previous step.

In [16]:
# Click the button below to check your work.
step14Complete = False

# Function to check if the student's answer was correct.
def step_14():
    # Important variables that must be accessed outside of this function.
    global step14Complete, result

    # Loading, in case the check is slow.
    with output14:
        output14.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput14.value == ""):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step14Complete = False

    else:
        # Escape single quotes by replacing them with '\''.
        escaped_user_input = re.sub(r"(\"|\')", r"'\''", userInput14.value.strip())
        
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_2.py 14 '{escaped_user_input}'" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo '{escaped_user_input}' > /home/.checker/responses/step_14_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if (result.returncode == 0):
            output14.clear_output()
            with output14:
                display(HTML("<span style='color: green;'>Your payload is correct! After sending your payload to the website, it produced the table below in your SQL database.</span>"))
                step14Complete = True
        
        elif (result.returncode == 1):
            output14.clear_output()
            with output14:
                display(HTML("<span style='color: red;'>Your payload didn't work. Try again.</span>"))
                step14Complete = False
        
        elif (result.returncode == 2):
            output14.clear_output()
            with output14:
                display(HTML("<span style='color: red;'>An error occurred while testing your payload. Please try again or contact your professor or TA.</span>"))
                step14Complete = False

        elif (result.returncode == 3):
            output14.clear_output()
            with output14:
                display(HTML("<span style='color: green;'>You have completed this step previously, and your file is patched.</span>"))
                step14Complete = True

        # Now, if the return code was 0 or 1, it returned a response. Print the HTML of the response:
        if (result.returncode == 0 or result.returncode == 1):
            result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_14_response.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
            with output14:
                if ("No results found. Please enter a valid ID." in result.stdout):
                    display(HTML("<br><p style='color: blue;'><em>Your php_practice.php file has its SQL injection vulnerability fixed.</em></p>"))
                
                else:
                    display(HTML("<br><p style='color: blue;'><em>Here is your SQL database after sending your payload:</em></p>"))
                    results = run_sql_query(userInput14.value)

                    if results:
                        markdown = create_html_table(results)
                        display(HTML(markdown))

def check_step_14(b):
    if (warn_student()):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_14()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("14", result.returncode, userInput14.value)

def reset_step_14(b):
    if (not step11Complete):
        output14.clear_output()
        with output14:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 14', shell=True, stdout=subprocess.DEVNULL)
    
        output14.clear_output()
        with output14:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))


# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading14 = widgets.Output()
display(loading14)
with loading14:
    loading14.clear_output()
    display(HTML("<span>Loading your saved response... <img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput14 = widgets.Text(
    placeholder='Type your payload here',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_14_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput14.value = result.stdout

# After the student's response was loaded, clear the output.
loading14.clear_output()

# Creating the button.
button = widgets.Button(description="Test Payload")

# Create an extra button for resetting the database.
button_step_14 = widgets.Button(description="Reset Database")

# Creating an output area.
output14 = widgets.Output()

# Run the command on click.
button.on_click(check_step_14)
button_step_14.on_click(reset_step_14)

# Display the output.
display(userInput14, button, button_step_14, output14)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type your payload here')

Button(description='Test Payload', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

## <strong>Topic 3: Prepared Statements</strong>

PHP has a built-in functionality to mitigate the risks of SQL injection.

### mysqli_real_escape_string() - <em style="color: red;">Unsafe</em>

First, there is a function called ```mysqli_real_escape_string()```, which is an <strong>unsafe</strong> sanitization function. 

This function requires a connection to be made with the SQL server before it can be called. This function takes any unsafe characters that can occur within a SQL statement, and safely escapes characters so that it treats SQL tokens as regular string characters. However, there are security vulnerabilities to this statement, which you can <a href="https://stackoverflow.com/questions/5741187/sql-injection-that-gets-around-mysql-real-escape-string">read about here</a>. 

To put simply, this function becomes unsafe if your database character set does not match your PHP character set. This means using double quotes for your SQL statement, but single quotes for your user input, or vice versa, can break your security.

### mysqli_prepare() - <em style="color: green;">Safe</em>

The safer alternative of ```mysqli_real_escape_string()``` is ```mysqli_prepare()```, which is a <strong>prepared statement</strong>.

A prepared statement is a pre-compiled query, which means that it's sent to the database before it takes input. Once it has been compiled, the user input is replaced by ```?``` within the query, and becomes impossible to inject SQL code into. Prepared statements take the input type for each ```?``` within the query, and their types get checked when compiling the code before replacing the ```?``` with the user's input.

Here is an example of a prepared statement:

```$stmt = $conn->prepare("INSERT INTO table (column1, column2) VALUES (?, ?)");```

Before a statement can be executed, you need to bind parameters to the ```?``` values in the prepared statement. This tells PHP which variable types are being assigned to the question marks in the statement. So, the next line would be:

```$stmt->bind_param('ss', $userInput1, $userInput2);```

The ```ss``` tells SQL that ```$userInput1``` is a string, as well as ```$userInput2```. Additionally, if any inputs are an ```i``` type, it tells SQL that you are sending an int, ```d``` is a float, and ```b``` is a blob. You only need to worry about ```i``` and ```s``` for this lab.

Finally, the last step is to run ```$stmt->execute();``` to execute the statement. If your SQL query returns a row from a table, it can be retrieved with ```$result = $stmt->get_result();```, then you can convert the result into an array with ```fetch_row``` or ```fetch_array```. Place this statement within a ```while``` loop if you would like to read all rows from the query.

### Step 15: Writing Your First Prepared Statement

Using the instructions above, turn the SQL statement in ```php_practice.php``` into a prepared statement. Once you have converted the SQL queries into prepared statements, you are encouraged to take your payloads from above and test your work.

To help direct you with what to do, you will need to do the following:
- Convert the ```$student_id``` in ```$sql``` with a ```?```.
- Replace ```$conn->multi_query($sql)``` with ```$conn->prepare($sql);``` and assign it to a variable, like ```$stmt```.
- Call ```$stmt->bind_param("i", $student_id)``` after the prepared statement.
- Execute the query with ```$stmt->execute()```.
- Replace ```$conn->store_result()``` with ```$stmt->get_result()```, then leave it assigned to ```$result```.

If you're stuck, the PHP documentation is very helpful with demonstrating prepared statements. Feel free to view these examples. <u>While reading these documentation pages, you must use the OOP approach in your answer. Not the procedural approach.</u> You're heavily encouraged to look at these two documentation pages for examples that relate to this question.
- <a href="https://www.php.net/manual/en/mysqli.prepare.php">mysqli_prepare</a>
- <a href="https://www.php.net/manual/en/mysqli-stmt.execute.php">mysqli_stmt_execute</a>

If you would like to revert the database to completing Step 14, you may click on the reset button. This will not reset your PHP file.

When you have finished writing your prepared statement, click on the "Check Work" button to test this step. A SQL injection will be tested on your database, which should fail.

<span style="color: red;"><strong><img src="resources/alert.png" style="width: 12px"> Warning:</strong></span> If you comment out any of your prepared statement lines, it will be marked as missing. If you miss any statements, the check for this step will tell you what function(s) you missed. Make sure that you use the OOP approach (with ```->``` symbols) instead of procedural when using documentation online.

In [17]:
# Click the button below to check your work.
step15Complete = False

# Function to check the permissions.
def step_15():
    # Important variables that must be accessed outside of this function.
    global step15Complete, result

    with output15:
        output15.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/section_3.py', shell=True, text=True, capture_output=True)

    if (result.returncode == 0):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: green;'>Success! SQL injection did not occur on your website.</span>"))
            step15Complete = True

    elif (result.returncode == 1):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>You are using the correct functions in your PHP file, but SQL injection still occurred.</span>"))
            step15Complete = False
    
    elif (result.returncode == 2):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>There is an error with this step. Either php_practice.php or SQL is not working. Please contact your professor or TA.</span>"))
            step15Complete = False

    elif (result.returncode == 3):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>You don't have all of the necessary functions used for a prepared statement. " + result.stdout + "</span>"))
            step15Complete = False

def check_step_15(b):
    if (warn_student()):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_15()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("15", result.returncode)

def reset_step_15(b):
    if (not step14Complete):
        output15.clear_output()
        with output15:
            display(HTML("<span style='color: red;'>You must complete the previous step before this button is enabled.</span>"))

    else:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/reset_db.py 15', shell=True, stdout=subprocess.DEVNULL)
   
        output15.clear_output()
        with output15:
            if (result.returncode == 0):
                display(HTML("<span>Your database has been reset.</span>"))
            elif (result.returncode == 1):
                display(HTML("<span>Something wrong occurred and your database wasn't reset. Please contact your professor/TA.</span>"))
            elif (result.returncode == 2):
                display(HTML("<span>A SQL error occurred. Please contact your professor/TA.</span>"))

# Creating the button.
button = widgets.Button(description="Test for SQL Injection")

# Create an extra button for resetting the database.
button_step_15 = widgets.Button(description="Reset Database")

# Creating an output area.
output15 = widgets.Output()

# Run the command on click.
button.on_click(check_step_15)
button_step_15.on_click(reset_step_15)

# Display the output.
display(button, button_step_15, output15)

Button(description='Test for SQL Injection', style=ButtonStyle())

Button(description='Reset Database', style=ButtonStyle())

Output()

## <strong>Topic 4: A Large-Scale Application of SQL Injection</strong>

For the final topic, you will be testing your knowledge on a website called FrobozzCo Community Credit Union (FCCU). The developers for FCCU have written the database without knowledge about SQL injection, and you will be responsible for attacking their website, draining someone's savings, then fixing their website.

Navigate to ```localhost:port/index.html``` to go to the bank's landing page. Once again, <strong>port forwarding is required</strong> in order to access the website from your browser. ```port``` is a value that you used in your SSH statement.

The source code for this site is located in ```/usr/lib/cgi-bin/FCCU.php```.

Earlier in the lab, you saw a database called ```fccu``` in MySQL. The website that you will break is doing to use this database. If you are currently signed into MySQL, type ```USE fccu;``` to switch databases. Inside of the database, you will be able to view the usernames and passwords of all accounts. You may try signing into any account to view what customers can view.

For this topic, you will not be shown the SQL query upon submitting the forum. You will need to think critically and understand how your input will be placed within the SQL statements.

### Step 16: Accessing Someone Else's Account

On the sign-in page, type a SQL statement into the ID field so that you can access anyone's account without knowing the ID numbers ahead of time. <u>You are not allowed to use any of the customer's ID numbers in the payload.</u>

When you have a working payload, copy/paste it below to check your work.

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>
- Consider using a similar attack to what you used in Topic 2.
- The ID field of the user's input is placed in the middle of a SQL statement. Consider using a SQL comment (```--```) to remove everything after your injection so that there is no error.
- When signing into accounts, the PHP file will take the first row from the SQL query then sign into that user. Therefore, when delivering your payload, the first row of your payload will be the target. <u>This will be important to know for the next step.</u>
- You must type something into the password field for the attack to work.
- Your browser may show you the input history when you enter something into the text field. If you forgot to copy your working payload, you can go back and click on the entry box to view past inputs.

In [18]:
# Click the button below to check your work.
step16Complete = False

# Function to check if the student's answer was correct.
def step_16():
    # Important variables that must be accessed outside of this function.
    global step16Complete, result

    # Loading, in case the check is slow.
    with output16:
        output16.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput16.value == ""):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step16Complete = False

    else:
        # Escape single quotes by replacing them with '\''.
        escaped_user_input = shlex.quote(userInput16.value.strip())
       
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_4.py 16 {escaped_user_input}" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo {escaped_user_input} > /home/.checker/responses/step_16_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
       
        if (result.returncode == 0):
            output16.clear_output()
            with output16:
                # Now, if the return code was 1, it returned the correct response. Print the HTML of the response:
                display(HTML("<span style='color: green;'>Your payload worked! Here is the output from your payload.</span>"))
                step16Complete = True
                result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_16_response.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
                display(HTML(result.stdout))

        # Student is using a three-digit number.
        elif (result.returncode == 3):
            output16.clear_output()
            with output16:
                display(HTML("<span style='color: red;'>It appears you may be using a customer's ID in your payload. That is, a number that's three digits or more. Please remove it and try a different approach.</span>"))
                step16Complete = False

        # Did not include a comment, which would not allow the student into the account.
        elif (result.returncode == 4):
            output16.clear_output()
            with output16:
                display(HTML("<span style='color: red;'>You did not use a SQL comment (--) in your payload. This is required for your payload to work.</span>"))
                step16Complete = False

        # Payload did not allow the student into someone's account.
        elif (result.returncode == 1):
            output16.clear_output()
            with output16:
                display(HTML("<span style='color: red;'>Your payload did not sign you into someone's account. Please try again.</span>"))
                step16Complete = False

        # Something else would've happened to cause this.
        elif (result.returncode == 2):
            output16.clear_output()
            with output16:
                display(HTML("<span style='color: red;'>An error occurred when testing your payload. Please contact your professor or TA.</span>"))
                step16Complete = False

def check_step_16(b):
    if (warn_student()):
        output16.clear_output()
        with output16:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_16()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("16", result.returncode, userInput16.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading16 = widgets.Output()
display(loading16)
with loading16:
    loading16.clear_output()
    display(HTML("<span>Loading your saved response... <img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput16 = widgets.Text(
    placeholder='Type your payload here',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_16_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput16.value = result.stdout

# After the student's response was loaded, clear the output.
loading16.clear_output()

# Creating the button.
button = widgets.Button(description="Test Payload")

# Creating an output area.
output16 = widgets.Output()

# Run the command on click.
button.on_click(check_step_16)

# Display the output.
display(userInput16, button, output16)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type your payload here')

Button(description='Test Payload', style=ButtonStyle())

Output()

### Step 17: Accessing Multiple Accounts

In the previous step, you found a way to access someone's account by using SQL injection. Now, on the sign-in page, type a SQL statement into the ID field so that you can access the next valid ID. <u>The output of your SQL injection cannot attack the user from the previous step.</u>

When you have a working payload, copy/paste it below to check your work. <u>You ARE allowed to use the ID from the previous step for your payload.</u>

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tip:</strong></span> This payload will be very similar to the previous one. However, consider using ```OFFSET```. Recall that this clause cannot be used without ```LIMIT```. If you prefer not to use these clauses, consider using another payload like ```id > number```, where ```number``` is the ID that you found from the previous user that you compromised.

In [19]:
# Click the button below to check your work.
step17Complete = False

# Function to check if the student's answer was correct.
def step_17():
    # Important variables that must be accessed outside of this function.
    global step16Complete, step17Complete, result

    # Loading, in case the check is slow.
    with output17:
        output17.clear_output()
        display(HTML("<span><img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

    # First, check to see if the field is empty.
    if (userInput17.value == ""):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'>You did not provide input for this step.</span>"))
            step17Complete = False

    # Additionally, check to see if the previous step was completed.
    # Required for the checker script! Will break if this step is completed correctly before Step 16 is attempted.
    elif (not step16Complete):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'>You must complete Step 16 before attempting this step.</span>"))
            step17Complete = False

    else:
        # Escape single quotes by replacing them with '\''.
        escaped_user_input = shlex.quote(userInput17.value.strip())
       
        # Construct the SSH command for testing the SQL query.
        sql_test_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_4.py 17 {escaped_user_input}" """
        result = subprocess.run(sql_test_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        # Construct the SSH command for saving the student's response.
        save_command = f"""ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "echo {escaped_user_input} > /home/.checker/responses/step_17_answer.txt" """
        save_result = subprocess.run(save_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
       
        if (result.returncode == 0):
            output17.clear_output()
            with output17:
                # Now, if the return code was 1, it returned the correct response. Print the HTML of the response:
                display(HTML("<span style='color: green;'>Your payload worked! Here is the output from your payload.</span>"))
                step17Complete = True

        # Student may be using a customer's ID (or not the previous customer's ID) in their payload.
        elif (result.returncode == 3):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>It appears you may be using a customer's ID in your payload. You may only use the ID used from the previous response. Please remove it and try a different approach.</span>"))
                step17Complete = False

        # Did not include a comment, which would not allow the student into the account.
        elif (result.returncode == 4):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>You did not use a SQL comment (--) in your payload. This is required for your payload to work.</span>"))
                step17Complete = False

        # Shouldn't happen, because this should've been caught previously.
        elif (result.returncode == 5):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>The previous step was not completed or your response was never saved. Please re-attempt Step 16. Contact your professor/TA if you need assistance.</span>"))
                step17Complete = False

        # SQLi was correct, but did not return another user from the previous step.
        elif (result.returncode == 6):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>Your attack was successful, but did not return a different user from the previous step. Here is the result of your payload. Try again.</span>"))
                step17Complete = False

        # Payload did not sign into a user's account.
        elif (result.returncode == 1):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>Your payload did not sign you into someone's account. Please try again.</span>"))
                step16Complete = False

        # Something else went wrong.
        elif (result.returncode == 2):
            output17.clear_output()
            with output17:
                display(HTML("<span style='color: red;'>An error occurred when testing your payload. Please contact your professor or TA.</span>"))
                step16Complete = False

        # If the return code is 1 or 6, then a response was successful, and can be displayed to the student.
        # Return code 5 also writes output, but the check cannot be finished because it cannot be compared to the previous step.
        # Shouldn't happen, but in case it does, you can check the step_16_response.txt file to see what's wrong.
        if (result.returncode == 0 or result.returncode == 6):
            with output17:
                result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_17_response.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
                display(HTML(result.stdout))

        
def check_step_17(b):
    if (warn_student()):
        output17.clear_output()
        with output17:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_17()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("17", result.returncode, userInput17.value)

# Retrieve the student's response. First, create a loading spinner, since this could take a second or two.
loading17 = widgets.Output()
display(loading17)
with loading17:
    loading17.clear_output()
    display(HTML("<span>Loading your saved response... <img width='14px' height='14px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))

# Creating a text area.
userInput17 = widgets.Text(
    placeholder='Type your payload here',
    description='Payload:',
    layout=widgets.Layout(width='90%')
)

# Checking if the step has been answered.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_17_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)
userInput17.value = result.stdout

# After the student's response was loaded, clear the output.
loading17.clear_output()

# Creating the button.
button = widgets.Button(description="Test Payload")

# Creating an output area.
output17 = widgets.Output()

# Run the command on click.
button.on_click(check_step_17)

# Display the output.
display(userInput17, button, output17)

Output()

Text(value='', description='Payload:', layout=Layout(width='90%'), placeholder='Type your payload here')

Button(description='Test Payload', style=ButtonStyle())

Output()

### Step 18: Wiping a Bank Account

Make some account (your choice) wire its total balance to the bank with routing number: ```314159265``` and account number: ```271828182845```

In [20]:
# Click the button below to check your work.
step18Complete = False

# Function to check the permissions.
def step_18():
    # Important variables that must be accessed outside of this function.
    global step18Complete, result

    with output18:
        output18.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/section_4.py 18 NA', shell=True)

    if (result.returncode == 0):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step18Complete = True

    elif (result.returncode == 1):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: red;'>A wire transfer to 271828182845 cannot be found.</span>"))
            step18Complete = False
    
def check_step_18(b):
    if (warn_student()):
        output18.clear_output()
        with output18:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_18()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("18", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output18 = widgets.Output()

# Run the command on click.
button.on_click(check_step_18)

# Display the output.
display(button, output18)

Button(description='Check Work', style=ButtonStyle())

Output()

### Step 19: Adding/Updating Accounts

Why you can't create a new account or arbitrarily update account balances within the username field?

This is a multiple choice question.

In [21]:
# Click the button below to check your work.
step19Complete = False

# Function to check if the student's answer was correct.
def step_19():
    # Important variables that must be accessed outside of this function.
    global step19Complete, result

    if (multchoice.value == None):
        output19.clear_output()
        with output19:
            display(HTML("<span style='color: red;'>You did not select an answer.</span>"))
            step19Complete = False

    else:
        # Checking the answer:
        result = subprocess.run('ssh -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "/home/.checker/section_4.py 19 \\"' + multchoice.value + '\\""', shell=True, text=True, capture_output=True)
        
        if (result.returncode == 0):
            output19.clear_output()
            with output19:
                display(HTML("<span style='color: green;'>Correct! " + result.stdout + "</span>"))
                step19Complete = True
    
        elif (result.returncode == 1):
            output19.clear_output()
            with output19:
                display(HTML("<span style='color: red;'>Double-check your answer. " + result.stdout + "</span>"))
                step19Complete = False
            
def check_step_19(b):
    if (warn_student()):
        output19.clear_output()
        with output19:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_19()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("19", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Answer")

# Creating an output area.
output19 = widgets.Output()

# Run the command on click.
button.on_click(check_step_19)

display(HTML("""
<style>
.custom-radio .widget-radio-box label {
    white-space: nowrap;
    overflow: hidden;
    text-overflow: ellipsis;
    display: block;
}
</style>
"""))

# Creating a text area.
multchoice = widgets.RadioButtons(
    options=['A SELECT statement cannot be nested with a INSERT or UPDATE statement.',
             'The ID field restricts how many characters you can type. You would max out the character limit.',
            'INSERT or UPDATE provides no output. So, the statement(s) cannot be used.',
            'You cannot have more than one query when executing statements in FCCU.php.'],
    description='Select your answer:',
    index=None,  # No initial selection
    layout=widgets.Layout(width='650px', height='120px', margin='10px')
)

# Getting a previously saved response, if any.
result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli "cat /home/.checker/responses/step_19_answer.txt 2> /dev/null"', capture_output=True, text=True, shell=True)

# Pre-setting the index to whatever the student has answered.
if (result.stdout == 'A SELECT statement cannot be nested with a INSERT or UPDATE statement.'):
    multchoice.index = 0

elif (result.stdout == 'The ID field restricts how many characters you can type. You would max out the character limit.'):
    multchoice.index = 1

elif (result.stdout == 'INSERT or UPDATE provides no output. So, the statement(s) cannot be used.'):
    multchoice.index = 2

elif (result.stdout == 'You cannot have more than one query when executing statements in FCCU.php.'):
    multchoice.index = 3

# Display the output.
display(multchoice, button, output19)

RadioButtons(description='Select your answer:', layout=Layout(height='120px', margin='10px', width='650px'), o…

Button(description='Check Answer', style=ButtonStyle())

Output()

### Step 20: Fixing the Vulnerability

For the final step of the lab, you will have to convert the following entries into prepared statements:

- The sign-in field of the website.
- The "wire funds" field.
- The "transfer money" field.
- The "withdraw cash" field.

To assist you with fixing these statements, every query that needs to be updated in ```FCCU.php``` will appear like this:

```
/*** Block #1: CONVERT TO PREPARED STATEMENT ***/
$query = "SQL statement here";
$result = $mysqli->query($query) or die($mysqli->error());
$row = $result->fetch_array();
/***********************************************/
```

Here's a template of a fixed statement:
```
/*** Block #1: CONVERT TO PREPARED STATEMENT ***/
$query = "Updated SQL statement here";
$stmt = $mysqli->prepare($query);
$stmt->bind_param("si", $string_param, $int_param);
$stmt->execute();
$result = $stmt->get_result() or die($mysqli->error());
$row = $result->fetch_row();
/***********************************************/
```

Note that you are switching from ```fetch_array()``` to ```fetch_row()``` when using the prepared statements. You cannot use ```fetch_array()``` on the statement object, a value returned by <a href="https://www.php.net/manual/en/mysqli.prepare.php">prepare()</a>. Instead, you need to use methods associated with the <a href="https://www.php.net/manual/en/class.mysqli-stmt.php">mysqli_stmt</a> object to fetch results. Here is some documentation on <a href="https://www.php.net/manual/en/mysqli-result.fetch-row.php">mysqli_fetch_row</a>, which is what you're using instead.

The sign-in field will be tested to ensure that SQL injection will not occur. Every other field will be checked, but not tested. Recall that the source code for this site is located in ```/usr/lib/cgi-bin/FCCU.php```. <u>You need to use ```sudo``` to write into this file.

<strong>Do not remove the comments that surround the code that needs to be edited.</strong> For the grader to function properly, these two lines need to remain around your answers. 

<span style="color: green"><strong><img src="resources/idea.png" style="width: 12px"> Tips:</strong></span>
- There are a total of 9 statements which needs to be converted. The template is provided to you, and you can copy/paste the code into ```FCCU.php```, then make adjustments to the ```$query``` string and the variables in ```bind_param```.
- Whenever a variable is wrapped around 'single quotes', that indicates you must bind a <u>string</u> parameter. Likewise, if a variable does not have 'single quotes' around it, bind an integer to it.
  - Inside of your SQL statement, use ```?``` for your string, not ```'?'```.
- Be aware of the number of parameters in your SQL statements. Blocks #1-#4 and #6 have two parameters, block #5 and #7 use three parameters, and blocks #8 and #9 only have one parameter.
- Some prepared statements that need to be made are completely identical to other blocks. If you're patching a SQL statement that appears familiar, you should consider copy/pasting previous fixes that you made.
- <u>When using the template above, be careful not to delete the original ```$query``` that you're attempting to patch.</u> If you accidentally do this, you can find a copy of ```FCCU.php``` inside of ```/tmp``` on the ```sqli``` node.

In [22]:
# Click the button below to check your work.
step20Complete = False

# Function to check the permissions.
def step_20():
    # Important variables that must be accessed outside of this function.
    global step20Complete, result

    with output20:
        output20.clear_output()
        display(HTML("<span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    result = subprocess.run('ssh -o StrictHostKeyChecking=no -i /home/USERNAME_GOES_HERE/.ssh/merge_key USERNAME_GOES_HERE@sqli /home/.checker/section_4.py 20 NA', shell=True, capture_output=True, text=True)

    if (result.returncode == 0):
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: green;'>Success! All prepared statements were fixed and a payload doesn't allow you into anyone's account.</span>"))
            step20Complete = True

    elif (result.returncode == 1):
        # This error code will produce output. Gather the output:
        error_message = result.stdout
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: red;'>FCCU.php is not fully secure. " + str(error_message) + " Try again.</span>"))
            step20Complete = False

    elif (result.returncode == 3):
        error_message = result.stdout
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: red;'>All prepared statements were fixed, but a SQL injection attack still occurs on the sign-in page. Check Block #1 and fix any error(s) with your prepared statement.</span>"))
            step20Complete = False

    elif (result.returncode == 2):
        error_message = result.stdout
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: red;'>Either FCCU.php is missing, or all nine segments of SQL queries cannot be found. You may need to reset your FCCU.php file. Did you alter a comment surrounding a SQL query?</span>"))
            step20Complete = False

    elif (result.returncode == 4):
        error_message = result.stdout
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: red;'>Your SQL statements are fixed, but the SQLi payload failed. Please contact your professor/TA.</span>"))
            step20Complete = False

def check_step_20(b):
    if (warn_student()):
        output20.clear_output()
        with output20:
            display(HTML("<span style='color: red;'><strong>WARNING:</strong> You have an autosaved lab that you have not yet loaded. If you would like to load your progress, click \"Load Lab\" at the top of the notebook. Otherwise, clicking on this button again will assume you're restarting the lab!</span>"))
    else:
        step_20()

        # Auto-save.
        if (not runAllSteps):
            trigger_save("20", result.returncode)

# Creating the button.
button = widgets.Button(description="Check Work")

# Creating an output area.
output20 = widgets.Output()

# Run the command on click.
button.on_click(check_step_20)

# Display the output.
display(button, output20)

Button(description='Check Work', style=ButtonStyle())

Output()

## <strong>Grading</strong>

To check your overall grade, click on the button below.

In [23]:
# Click the button below to check your overall grade.
steps_to_check = [
    step_1, step_2, step_3, step_4, step_5, step_6, step_7, step_8, 
    step_11, step_12, step_13, step_14, step_15, step_16, step_17, 
    step_18, step_19, step_20
]

# Function to calculate grade after refreshing the cell.
def calculate_grade(b):
    # To not auto-save at each step.
    global runAllSteps
    runAllSteps = True

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML("<span>Testing all steps. Please wait.</span> \
            <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
    
    # Required for checking the boolean values.
    for func in steps_to_check:
        func()  # Call each function in order.

    # The array that stores all of the steps' results.
    steps = [
        step1Complete, step2Complete, step3Complete, step4Complete, step5Complete, 
        step6Complete, step7Complete, step8Complete, step11Complete, step12Complete, 
        step13Complete, step14Complete, step15Complete, step16Complete, step17Complete, 
        step18Complete, step19Complete, step20Complete
    ]

    output = ""
    stepsCorrect = 0
    total_steps = 20  # Total number of steps including optional steps

    for i in range(1, total_steps + 1):
        
        # Step 9 is optional.
        if i == 9:
            output += "<div style='color: orange;'>Step 9 is optional, and does not count towards your final grade.</div>"
            continue

        # Step 10 is a reading.
        if i == 10:
            output += "<div style='color: black;'>Step 10 is a reading section.</div>"
            continue

        # Check the corresponding step in the steps list.
        steps_index = i - 1
        if i > 10:
            steps_index -= 2  # Adjust index for steps 11-20.

        if steps[steps_index]:
            stepsCorrect += 1
            output += f"<div style='color: green;'>Step {i} is complete.</div>"
        else:
            output += f"<div style='color: red;'>Step {i} is incomplete.</div>"

    # The number of steps that count towards the final grade.
    stepsCorrect += 2  # Steps 9 and 10 are optional, and are considered correct no matter what.

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(total_steps) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

    # Allow each step to be auto-graded again.
    runAllSteps = False

# Create a button to refresh the cell and another to calculate grade.
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions.
grade_button.on_click(calculate_grade)

# Output area.
gradeOutput = widgets.Output()

# Display the buttons and output.
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()

### Stopping the Lab

Once you are done with the lab, click on the "Stop Lab" button below. <strong>This will delete your materialization, which will delete all of the lab's resources.</strong> Your progress is saved automatically in ```saves/``` within the sidebar of your XDC. You may load this lab in the future by clicking "Load Lab" at the top.

In [24]:
# Click the button below to stop the experiment.
def stoplab(button):
    # Check to make sure that the student wants to confirm ending the lab.
    if (confirm.value == False):
        with stop_output:
            stop_output.clear_output()
            display(HTML("<newline><span style='color: red;'>Please confirm that you wish to end the lab.</span>"))

    else:
        # Defining the lab name.
        labname = "sqli"
    
        # Writing the information to an empty field below the button.
        with stop_output:
            stop_output.clear_output()
            
            display(HTML("<span>Stopping the " + labname + " lab. This will take a minute to process. Please wait.</span> \
                <span><img width='12px' height='12px' style='margin-left: 3px;' src='resources/loading.gif'></span>"))
            stopexp = subprocess.run('su - USERNAME_GOES_HERE -c "bash /share/stopexp ' + labname + 'jup"', capture_output=True, text=True, shell=True)
            stop_output.clear_output()
            display(HTML("<span>Done. Result:</span>"))
            print(stopexp.stdout)
    
            display(HTML("<newline><span style='color: green;'><strong>Your lab has been ended.</strong></span>"))

# Creating the button.
stopButton = widgets.Button(description="Stop Lab")

# Create a confirmation check.
confirm = widgets.Checkbox(
    value=False,
    description='Confirm',
    disabled=False,
    indent=False
)

# Creating an output area.
stop_output = widgets.Output()

# Run the command on click.
stopButton.on_click(stoplab)

# Display the output.
display(confirm, stopButton, stop_output)

Checkbox(value=False, description='Confirm', indent=False)

Button(description='Stop Lab', style=ButtonStyle())

Output()